In [1]:
import sys
from Application.utils import show_images
from ClusteringManager import ClusterManager
import pickle
import numpy as np
import struct
import socket
import cv2
from PIL import Image
import traceback
import matplotlib.pyplot as plt
import os 
import time
host = "0.0.0.0"
port = 8080

In [2]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(3)
s.bind((host, port))
s.listen(10)

def capture_frame(connection):
    data = bytearray()
    message_size = struct.calcsize("L")
    connection_valid = True

    while True:
        while len(data) < message_size:
            packet = connection.recv(4096)
            if packet == b'': 
                connection_valid = False
                break
            else: 
                data += packet
        
        if connection_valid is False:
            break
        
        data, packed_msg_size = data[message_size:], data[:message_size]
        msg_size = struct.unpack("L", packed_msg_size)[0]
        
        while len(data) < msg_size:            
            packet = connection.recv(4096)
            if packet == b'': 
                connection_valid = False
                break
            else: 
                data += packet
        
        if connection_valid is False:
            break

        data, frame_data = data[msg_size:], data[:msg_size]
        frame = pickle.loads(frame_data)
        yield frame

def get_frame(path):
    with open(f"{path}", "rb") as fh:
        return pickle.load(fh)

def read_frame(path, iter=-1, sim_latency=0.45):

    files = os.listdir(path)
    
    count = 0
    while iter == -1 or count < iter:

        for file in files:
            time.sleep(sim_latency)
            yield get_frame(f"{path}/{file}").astype(np.uint8)

        count += 1


In [3]:

if True:
    saved_imgs = os.listdir("MultiClassTraining")
    max_ = 0
    for file in saved_imgs:
        name = file.replace(".png", "")
        if max_ < int(name):
            max_ = int(name)

    global_image_count = max_
    for frame in read_frame("Application/images"):

        # Update ClusterManager to use proper image dimenions for centroid initlization
        if ClusterManager.width is None:
            h, w, color_dim = frame.shape
            ClusterManager(w, h, min_area=0)

        # Threshold and retrieve contours, filtered by min and max contour area
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        _, threshed = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)
        contours, drawable = ClusterManager.filter_contours(threshed, contour_mode=cv2.RETR_LIST)

        # Compute K-Means clustering algorithm
        final_centroids = ClusterManager.kmeans(contours)

        # Compute sensor guesses from K-Means result on filtered image contours
        points_of_interest = ClusterManager.capture_subimages(frame, final_centroids)
        
        for image in points_of_interest:
            cv2.imwrite(f"MultiClassTraining/{global_image_count}.png", image)
            global_image_count += 1

else:
    for iter in range(10):
        try:
            connection, address = s.accept()
            count = 0
            for frame in capture_frame(connection):
                with open(f"Application/images/{count}", "wb") as fh:
                    pickle.dump(frame, fh, pickle.HIGHEST_PROTOCOL)
                count += 1
                cv2.imshow("img", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
            cv2.destroyAllWindows()
        except:
            time.sleep(5)
            print(traceback.format_exc())
            print(f"========================({iter})=======================")


KeyboardInterrupt: 